In [1]:
'''Next Steps 04.02.2018

Decision Tree:
- A conditional statement needs to be added to the Feature/Target objects.  
- If Prediction_type = Known_dataset, then use the current set up.
- else: 
- X_test will need to be replaced with the features from our unknown dataset (the master docketsheet)
- So we'll need to drop the first and last columns.  
- The function should then return a dataframe with the correction predictions. 

Also:
- We need to create an option in the code to include or not include the Dataframe_test.  
- This will need to be linked to the above decision tree conditional statement. 
- This way we can have an option to train the model and once we get the best set of hyperparameters
run the model to make a prediction on the unknown dataset. 

Question:
- How do we set up our code such that we only run stp3B when we want to?

'''


"Next Steps 04.02.2018\n\nDecision Tree:\n- A conditional statement needs to be added to the Feature/Target objects.  \n- If Prediction_type = Known_dataset, then use the current set up.\n- else: \n- X_test will need to be replaced with the features from our unknown dataset (the master docketsheet)\n- So we'll need to drop the first and last columns.  \n- The function should then return a dataframe with the correction predictions. \n\nAlso:\n- We need to create an option in the code to include or not include the Dataframe_test.  \n- This will need to be linked to the above decision tree conditional statement. \n- This way we can have an option to train the model and once we get the best set of hyperparameters\nrun the model to make a prediction on the unknown dataset. \n\nQuestion:\n- How do we set up our code such that we only run stp3B when we want to?\n\n"

In [2]:
# IMPORT LIBRARIES

In [1]:
import os
import pandas as pd

In [4]:
# IMPORT MODULES

In [5]:
os.chdir(r'/home/ccirelli2/Desktop/Docket-Sheet-Classification-v2/Modules')
import Step1_Module_Ngrams_FreqDist_version4_Ngrams as stp1
import Step2_P2_Module_Get_Top_Words_version4_Ngrams as stp2
import Step3_Module_Ngrams_Docketsheet_KeyWord_FreqDist as stp3
import Step4_Module_Machine_Learning_Algorithms as stp4
import Step5_Module_Measure_DocketSheetEntries_NoMatches as stp5_nomatch
import Step5_Module_Measure_Dependencies_Between_Stages as stp5_dependencies

In [6]:
# IMPORT NGRAM RESULTS:  CALCULATION = AVERAGE APPEARANCE

In [7]:
os.chdir(r'/home/ccirelli2/Desktop/Docket-Sheet-Classification/All_Results/Result_Ngrams')
Nograms_appearance = pd.read_excel(r'Docketsheet_FreqDist_Nograms_Average_appearance.xlsx')
Bigrams_appearance = pd.read_excel(r'Docketsheet_FreqDist_Bigrams_Average_appearance.xlsx')
Trigrams_appearance = pd.read_excel(r'Docketsheet_FreqDist_Trigrams_Average_appearance.xlsx')
Quadgrams_appearance = pd.read_excel(r'Docketsheet_FreqDist_Quadgrams_Average_appearance.xlsx')

In [8]:
# IMPORT NGRAM RESULTS:  CALCULATION = FREQUENCY

In [9]:
Nograms_frequency = pd.read_excel(r'Docketsheet_FreqDist_Nograms_Frequency_distribution.xlsx')
Bigrams_frequency = pd.read_excel(r'Docketsheet_FreqDist_Bigrams_Frequency_distribution.xlsx')

In [10]:
# IMPORT DOCKETSHEET WITH PRE-CLASSIFIED STAGES

In [11]:
os.chdir(r'/home/ccirelli2/Desktop/Docket-Sheet-Classification/Data_Files_applicable_all_code')
Docketsheet_train = 'DocketSheet Classification_70_02.22.2018.xlsx'

In [12]:
# IMPORT MASTER DOCKETSHEET WITH NON-CLASSIFIED STAGES

In [13]:
# Only run if the document has yet to be created
'''
File_masterdocketsheet = r'Master_Docket_sheets.xlsx'
Destination_loc = r'/home/ccirelli2/Desktop/Docket-Sheet-Classification/Data_Files_applicable_all_code'
df_transformed = stp3.transform_Master_Docketsheet(File_masterdocketsheet, Write2Excel =True, 
                                               destination_location = Destination_loc)
'''
os.chdir(r'/home/ccirelli2/Desktop/Docket-Sheet-Classification/Stage5_Predictions_Results')
Docketsheet_test = r'Master_Docketsheet_Chunk4.xlsx'


In [14]:
# CHUNK MASTER DOCKETSHEET
def chunk_master_docketsheetfile(Excel_file):

    df = pd.read_excel(Excel_file)
    df_chunked = df.iloc[150001:200000,:]

    # Write to Excel Module
    stp4.write_to_excel(    
                dataframe = df_chunked, 
                location = r'/home/ccirelli2/Desktop/Docket-Sheet-Classification/Stage5_Predictions_Results',
                filename = 'Master_Docketsheet_Chunk4')

    return df_chunked


In [15]:
# DRIVER FUNCTION

In [16]:
def MasterFunction( TrainTestMode, # 04.03.2018: This refers to stp3A & B. 
    
                    stp2_FreqDist_file, 
                        stp2_Calculation_meth, stp2_methodology_top_words, 
                        stp2_write2excel, stp2_destination_location, 
                        stp2_Ngram_type,
                   
                   stp3A_Docketsheet, 
                        stp3A_DirNgramLoc, stp3A_Iterable, 
                        stp3A_KeyPhrase, stp3A_Destination_location,
                        stp3A_Transpose4mlModel, stp3A_Write2Excel,
                   
                   stp3B_Docketsheet, 
                        stp3B_DirNgramLoc, stp3B_Iterable, 
                        stp3B_KeyPhrase, stp3B_Destination_location,
                        stp3B_Transpose4mlModel, stp3B_Write2Excel,
                   
                   stp4_Max_Depth, 
                        stp4_TrainTest, 
                        stp4_Metric):
    
    # BODY OF FUNCTION STARTS HERE:_____________________________________________________________________
    
    # STEP2: GENERATE KEY NGRAMS____________________________________________________:
    
    print('Step2: Generating our Key Ngrmas...\n')
    TopWords_Toggle_Calc_Methodology = stp2.get_top_words_toggle_methodology(
                    # Features
                    stp2_FreqDist_file, 
                    stp2_Calculation_meth, 
                    stp2_methodology_top_words, 
                    stp2_write2excel, 
                    stp2_destination_location, 
                    stp2_Ngram_type)
    # If writing to Excel, revert back to main dir where docketsheet is located. 
    os.chdir(r'/home/ccirelli2/Desktop/Docket-Sheet-Classification/Data_Files_applicable_all_code')
    
    
    # STEP3_A: DOCKETSHEET NGRAM APPEARANCE - TRAINING MODE_________________________:
    
    # Added 04.03.2018: Toggle Between Training & Test Modes
    if TrainTestMode == 'Training_Mode_Known_Data':
        print('Step3_A: Generating the Ngram matches for the \'Training\' Docketsheet entries...\n')
        stp3A_KeyNgram_Appearance_Training_DocketSheetEntries = stp3.get_DocketSheet_KeyWord_Appearance_Master(
                    # Features
                    stp3_Docketsheet                 = stp3A_Docketsheet, 
                    stp3_DirNgramLoc                 = stp3A_DirNgramLoc, 
                    stp3_Iterable                    = stp3A_Iterable, 
                    stp3_KeyPhrase                   = stp3A_KeyPhrase, 
                    stp3_Destination_location        = stp3A_Destination_location,
                    stp3_Transpose4mlModel           = stp3A_Transpose4mlModel, 
                    stp3_Write2Excel                 = stp3A_Write2Excel, 
                    stp3_Target_file                 = TopWords_Toggle_Calc_Methodology)
       
        # Step4: Decision Tree Prediction
        print('Step4:  Generating Decision Tree prediction\n')
        DecisionTreePrediction = stp4.simple_decision_tree(
                    Max_Depth                        = stp4_Max_Depth, 
                    TrainTest                        = stp4_TrainTest, 
                    Metric                           = stp4_Metric,
                    Docketsheet_known                = stp3A_KeyNgram_Appearance_Training_DocketSheetEntries,
                    Docketsheet_unknown              = None, # None as it will not be used in Training mode. 
                    TrainTestMode                    = TrainTestMode) #added 04.03.2018
    
        # Define File Name for In-Memory Reference (Not reading from a file + path name)
        df_inmemory_name = str(stp2_Ngram_type + '_' + stp2_Calculation_meth + '_' + 
        stp2_methodology_top_words + '_' + stp4_Metric + '_' + 'Depth' + '_' + str(stp4_Max_Depth))
    
        # Calculate the number of docket sheet rows without a match.  Send to user via print. 
        Count_zero = stp5_nomatch.get_count_nomatch_columns(
            File = stp3A_KeyNgram_Appearance_Training_DocketSheetEntries, Count = 'Count_zero')
        Count_total = stp5_nomatch.get_count_nomatch_columns(
            File = stp3A_KeyNgram_Appearance_Training_DocketSheetEntries, Count = 'Count_all')
    
        # Calculate the extent of dependencies between the KeyWords in Each Stage. 
        Perct_dependency = stp5_dependencies.measure_dependencies(TopWords_Toggle_Calc_Methodology)
    
        # OTHER PERFORMANCE METRICS REGARDING THE MODEL
        print('Other performance metrics:')
        print('\tPercent Dependency:   The Ngram Key Selection resulted in =>' + ' ' + str(Perct_dependency) + ' of overlap (dependency) between the key words')
        print('\tPercent None-Matches: In addition, this selection resulted in => ', round(Count_zero/Count_total, 2)*100, ' ' + 
          'percentage of Docketsheet rows without a matching Ngram', '\n')
    
        return DecisionTreePrediction
    
    
    # STEP3 A&B - TESTING MODE________________________________________________________________
    
    elif TrainTestMode == 'Prediction_Mode_Unknown_Data/':
        print('Step3_A: Generating the Ngram matches for the \'Training\' Docketsheet entries...\n')
        stp3A_KeyNgram_Appearance_Training_DocketSheetEntries = stp3.get_DocketSheet_KeyWord_Appearance_Master(
                    # Features
                    stp3_Docketsheet                 = stp3A_Docketsheet, 
                    stp3_DirNgramLoc                 = stp3A_DirNgramLoc, 
                    stp3_Iterable                    = stp3A_Iterable, 
                    stp3_KeyPhrase                   = stp3A_KeyPhrase, 
                    stp3_Destination_location        = stp3A_Destination_location,
                    stp3_Transpose4mlModel           = stp3A_Transpose4mlModel, 
                    stp3_Write2Excel                 = stp3A_Write2Excel, 
                    stp3_Target_file                 = TopWords_Toggle_Calc_Methodology)
    
        # Added on 03.31.2018:  Step_3 B was added. 
        print('Step3_B: Generating the Ngram matches for the \'Test\' Docketsheet entries...\n')
        stp3B_KeyNgram_Appearance_Test_DocketSheetEntries = stp3.get_DocketSheet_KeyWord_Appearance_Master(
                    # Features
                    stp3_Docketsheet                 = stp3B_Docketsheet, 
                    stp3_DirNgramLoc                 = stp3B_DirNgramLoc, 
                    stp3_Iterable                    = stp3B_Iterable, 
                    stp3_KeyPhrase                   = stp3B_KeyPhrase, 
                    stp3_Destination_location        = stp3B_Destination_location,
                    stp3_Transpose4mlModel           = stp3B_Transpose4mlModel, 
                    stp3_Write2Excel                 = stp3B_Write2Excel, 
                    stp3_Target_file                 = TopWords_Toggle_Calc_Methodology)
    
        # Step4: Decision Tree Prediction
        print('Step4:  Generating Decision Tree prediction\n')
        DecisionTreePrediction = stp4.simple_decision_tree(
                    Max_Depth                        = stp4_Max_Depth, 
                    TrainTest                        = stp4_TrainTest, 
                    Metric                           = stp4_Metric,
                    Docketsheet_known                = stp3A_KeyNgram_Appearance_Training_DocketSheetEntries,
                    Docketsheet_unknown              = stp3B_KeyNgram_Appearance_Test_DocketSheetEntries,
                    TrainTestMode                    = TrainTestMode) #added 04.03.2018
       
        return DecisionTreePrediction 

In [17]:
# NOTES

In [18]:
'''
Note that you the ytrain and test objects likely contain the arrays and attributes that we are calling like confusion matrix 
and or report are likely just ways to arrange these predictions. 
**** Note that you created a new object in Step4, the decision tree model, that allows for you to return a dataframe of the 
individual predictions.  This function has dependencies, in particular the second function that generates the Accuracy, 
Matrics, etc scores.  You need to find a way to stop the second function from running and to only pass the data frame 
object to the user. 

'''

'\nNote that you the ytrain and test objects likely contain the arrays and attributes that we are calling like confusion matrix \nand or report are likely just ways to arrange these predictions. \n**** Note that you created a new object in Step4, the decision tree model, that allows for you to return a dataframe of the \nindividual predictions.  This function has dependencies, in particular the second function that generates the Accuracy, \nMatrics, etc scores.  You need to find a way to stop the second function from running and to only pass the data frame \nobject to the user. \n\n'

In [19]:
# RUN FUNCTION

In [20]:
KeyNgrams = MasterFunction(
                    
                    TrainTestMode                  = 'Prediction_Mode_Unknown_Data',
                        
                    stp2_FreqDist_file             = Nograms_frequency,             # In general Freq seems to perf better. 
                        stp2_Calculation_meth      = 'CalculationI_homebrew_STDV', 
                        stp2_methodology_top_words = 'Top15_highest_STDV', 
                        stp2_write2excel           = False, 
                        stp2_destination_location  = '/home/ccirelli2/Desktop/', 
                        stp2_Ngram_type            = 'Nograms', 

                    stp3A_Docketsheet               = Docketsheet_train, 
                        stp3A_DirNgramLoc           = None, 
                        stp3A_Iterable              = False, 
                        stp3A_KeyPhrase             = None, 
                        stp3A_Destination_location  = None,
                        stp3A_Transpose4mlModel     = True, 
                        stp3A_Write2Excel           = False, 

                    stp3B_Docketsheet               = Docketsheet_test, 
                        stp3B_DirNgramLoc           = None, 
                        stp3B_Iterable              = False, 
                        stp3B_KeyPhrase             = None, 
                        stp3B_Destination_location  = None,
                        stp3B_Transpose4mlModel     = True, 
                        stp3B_Write2Excel           = False,
    
                    stp4_Max_Depth                  = 13, 
                        stp4_TrainTest              = 'Train', 
                        stp4_Metric                 = 'Accuracy')              

Step2: Generating our Key Ngrmas...

Step3_A: Generating the Ngram matches for the 'Training' Docketsheet entries...

	25% completed
	50% completed
	75% completed

Step3_B: Generating the Ngram matches for the 'Test' Docketsheet entries...

	25% completed
	50% completed
	75% completed

Step4:  Generating Decision Tree prediction

Generating prediction for the unknown dataset

Returning prediction for the unknown dataset: 



In [21]:
KeyNgrams

array([ 5.,  5.,  5., ...,  1.,  1.,  1.])

In [22]:
# Append Results to File 

In [23]:
# Create a Dataframe from the predictions

def create_dataframe_write2Excel(Array, Docketsheet_test):
    
    os.chdir(r'/home/ccirelli2/Desktop/Docket-Sheet-Classification/Stage5_Predictions_Results')
    
    # Read the Docketsheet Test file into memory as a df
    df_docketsheet = pd.read_excel(Docketsheet_test)
    # Limit dataframe to match what was input into the ML model. 
    delimiter = [isinstance(x,str) for x in df_docketsheet['docket_text']]
    df_limited = df_docketsheet[delimiter]
    df_limited['Prediction'] = Array
    
    print(df_limited.head())
    stp4.write_to_excel(dataframe = df_limited, 
                    location = r'/home/ccirelli2/Desktop/Docket-Sheet-Classification/Stage5_Predictions_Results',
                    filename = 'Step5_Prediction_Appended_Docketsheet_Chunk4')
    
    return None
    

In [24]:
df_prediction = create_dataframe_write2Excel(KeyNgrams, Docketsheet_test)

/home/ccirelli2/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


              case_number activity_date activity_number  \
150001  1:15-cv-01452-WBH    2016-02-05              55   
150002  1:15-cv-01452-WBH    2016-02-09              56   
150003  1:15-cv-01452-WBH    2016-02-18              57   
150004  1:15-cv-01452-WBH    2016-02-23              58   
150005  1:15-cv-01452-WBH    2016-03-03              59   

                                              docket_text  document_try_flag  \
150001  NOTICE of Appearance by Steven Wright Likens o...                  0   
150002  MOTION for Application of the Court's Recent R...                  0   
150003  RESPONSE in Opposition re MOTION for Applicati...                  0   
150004  REPLY BRIEF re (54 in 1:15-cv-02386-WBH) MOTIO...                  0   
150005  NOTICE to Take Deposition of Gentiva 30(b)(6) ...                  0   

        document_flag  document_link document_text state  district  \
150001              0            NaN           NaN    GA  Northern   
150002              0   

In [20]:
def transform_rowlvl_2_caselvl(File):
    # Change directory
    os.chdir(r'/home/ccirelli2/Desktop/Docket-Sheet-Classification-v2/Stage5_Predictions_Results')
    
    # Read File into memory as a dataframe
    df = pd.read_excel(File)
    # Delimit dataframe
    df_limited = df[['case_number', 'Prediction', 'activity_date']]
    
    # Create a Dictionary, Key = Case Number, Value = Stage. 
    Dict = {}
    # Create a counter to identify the first row of our dataframe. 
    Count = 0
    # Create a list to capture the stages per case. 
    List = []
    # Iterate over each row of the dataframe. 
    for x,y in zip(df_limited.itertuples(), df_limited[1:].itertuples()):
        # Add to our counter over each iteration. 
        Count += 1
        # If Count == 1, append to the list the first row's prediction value. 
        if Count == 1:
            List.append(x[2])
        # If count > 1
        else:
            # If the second row case num is equal to the first row case num, keep appending the values. 
            if y[1] == x[1]:
                List.append(y[2])
            # At which point the next rows case number does not equal the preceding one, append the List
            else:
                # Create our dictiony key == case number and value == List of stages for the given case. 
                Dict[x[1]] = set(List)
                List = []
      
    # Transform Dictionary 
    Dict_new = {}
    for key in Dict.keys():
        List = []
        for num in range(1,12):
            if num in Dict[key]:
                List.append(1)
            else:
                List.append(0)
        Dict_new[key] = List
    
    # Create Dataframe
    df_final = pd.DataFrame(Dict_new).transpose()
       
    df_final = df_final.rename(index =str, columns = {0:"Stage1", 
                                                      1:"Stage2", 
                                                      2:"Stage3", 
                                                      3: "Stage4", 
                                                      4:"Stage5", 
                                                      5:"Stage6", 
                                                      6:"Stage7", 
                                                      7:"Stage8", 
                                                      8:"Stage9", 
                                                      9:"Stage10", 
                                                      10:"Stage11"})
    print(df_final.head())
    
    stp4.write_to_excel(dataframe = df_final, 
                    location = r'/home/ccirelli2/Desktop',
                    filename = 'Case_Level_Predictions_Chunk1')
    
    # Return the dictonary to the user
    return None
        

In [21]:
df_transformed = transform_rowlvl_2_caselvl(r'Step5_Prediction_Appended_Docketsheet_Chunk1.xlsx')

                   Stage1  Stage2  Stage3  Stage4  Stage5  Stage6  Stage7  \
1:10-cv-00002-ODE       1       1       1       0       1       0       0   
1:10-cv-00007-JEC       0       1       0       1       1       1       1   
1:10-cv-00012-TWT       1       0       1       1       1       0       1   
1:10-cv-00017-JOF       0       1       1       1       1       1       0   
1:10-cv-00018-RLV       0       1       1       1       1       0       0   

                   Stage8  Stage9  Stage10  Stage11  
1:10-cv-00002-ODE       1       0        0        1  
1:10-cv-00007-JEC       0       0        0        1  
1:10-cv-00012-TWT       0       0        0        1  
1:10-cv-00017-JOF       1       0        0        1  
1:10-cv-00018-RLV       1       0        0        1  


In [ ]:
{'1:15-cv-01452-WBH': {(5, Timestamp('2016-03-01 00:00:00')), 
                       (5, Timestamp('2016-02-18 00:00:00')), 
                       (4, Timestamp('2016-03-16 00:00:00')), 
                       (5, Timestamp('2016-03-03 00:00:00')), 
                       (7, Timestamp('2016-03-23 00:00:00')), 
                       (5, Timestamp('2016-04-29 00:00:00')), 
                       (5, Timestamp('2016-02-23 00:00:00')), 
                       (8, Timestamp('2015-07-20 00:00:00')), 
                       (5, Timestamp('2016-03-23 00:00:00')), 
                       (5, Timestamp('2016-02-05 00:00:00')), 
                       (5, Timestamp('2015-11-18 00:00:00')), 
                       (5, Timestamp('2016-03-14 00:00:00')), 
                       (4, Timestamp('2017-06-22 00:00:00')), 
                       (5, Timestamp('2016-03-08 00:00:00')), 
                       (8, Timestamp('2017-06-08 00:00:00')), 
                       (5, Timestamp('2016-03-15 00:00:00')), 
                       (5, Timestamp('2015-06-30 00:00:00')), 
                       (11, Timestamp('2017-06-22 00:00:00')), 
                       (8, Timestamp('2017-06-07 00:00:00')), 
                       (5, Timestamp('2016-03-10 00:00:00'))}